In [ ]:
# Imports
!pip install gTTS
from gtts import gTTS
from IPython.display import Audio
import math
import urllib.request
import random
import requests
from IPython.core.display import display, HTML
from bs4 import BeautifulSoup
from datetime import date, datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
np.set_printoptions(suppress=True)
print("### Imports ready ###")

In [ ]:
# Read input data from Andruino
# data set 1
# file_url = 'https://drive.google.com/uc?export=download&id=1cgMSwwPIxhbfYRXOR6lYpKT0AQn38nG4'
# data set 2
file_url = 'https://drive.google.com/uc?export=download&id=1tZm99VmfUpSQi9QpEktL3xjeubDZRMX2'
file_name = 'data.csv'
input_file = urllib.request.urlretrieve(file_url, 'data.csv')
sensors_data = np.genfromtxt(file_name, delimiter=',')
tstp_data = sensors_data[:, 0]
temp_data = sensors_data[:, 1]
hum_data = sensors_data[:, 2]
pm_data = sensors_data[:, 3]

# Presentation
print("Input data")
print("We have taken input data from the sensors in Andruino during a day.")
print("The measures are taken in between 30 second lapses, currently we have a set of " + str(len(sensors_data)) + " measures.")
print("To test the expected results we have used gas near the sensors to provoke changes in the measures.")
print("Now we will start looking at each one of the sensors' set of data.")
print()

In [ ]:
# Temperature Analysis
print("Temperature: (taken in celsius grades)")
print()

print("Raw data:")
print(temp_data)
print()

temp_mean = round(np.mean(temp_data), 2)
print("Mean: " + str(temp_mean))
print()

temp_std = round(np.std(temp_data), 2)
print("Standard Deviation: " + str(temp_std))
print()

print("Observations:")
print("from what we can see in the data, the temperature didn't change much.")
print("This is expectable considering that the sensor is located inside a house, the temperature should be relarively stable.")
print()

print("Visualization:")

df = pd.DataFrame({
      'data': temp_data,
      'mean': [temp_mean for i in range(len(temp_data))],
      'std': [np.std(temp_data) for i in range(len(temp_data))]})

df.plot()
plt.show()

In [ ]:
# Relative Humidity Analysis
print("Relative Humidity:")
print("The dust sensor does not take good data when the humidity is above 95%.")
print("Here we should trigger an alarm if that situation happens to let the user know that the particules messure is not working.")

print("Raw data:")
print(hum_data)
print()

hum_mean = round(np.mean(hum_data), 2)
print("Mean: " + str(hum_mean))
print()

hum_std = round(np.std(hum_data), 2)
print("Standard Deviation: " + str(hum_std))
print()

print("Observations:")
print("The relative humedity vary during the day because of many reasons.")
print("For example, open a windows, turn on heaters or Air Conditioners.")
print("In our data we see that the RH didn't change much during the measures, but we have to be alert from humidiy not rising 95%, "+
      "above that PM measures will be inconsistence and it is a risk for the user.")
print()

print("Visualization:")
print("Mean and Standar Deviation")
df = pd.DataFrame({
      'data': hum_data,
      'mean': [hum_mean for i in range(len(hum_data))],
      'std': [np.std(hum_data) for i in range(len(hum_data))]})

df.plot()
plt.show()
print()

print("Histogram of values")
hum_data_rounded = np.around(hum_data)

plt.hist(hum_data_rounded, bins=range(int(min(hum_data_rounded)), int(max(hum_data_rounded))))
plt.show()

In [ ]:
# Particulate Matter Raw Analysis
print("Particulate Matter (PM):")
print("We will be focusing on measuring levels of particulate matter (PM) in the immediate area of the device using an Arduino dust sensor.")
print("The Arduino dust sensor collects counts of particles greater than 1 micron in size as air flows through the sensor.")
print("From those particle counts, we will have to estimate the total mass of particles present, as mass is the official reported standard for particulate matter used by the EPA.")
print("From those masses, we will  be able to determine the Air Quality Index (AQI) measurement for particulate matter.")
print("To calibrate our values, we will take outdoor samples and compare them to official  outdoor measurements reported by the EPA.")
print("There are some concerns with this method, as particulate matter is heavily affected by local sources (roads, restaurants, etc).")
print("But believe that these samples, along  with indoor tests looking at PM next to gas stoves, will provide a reasonable estimate for this proof of concept.")
print()

print("Raw data:")
print(pm_data)
print()

pm_mean = round(np.mean(pm_data), 2)
print("Mean: " + str(pm_mean))
print()

pm_std = round(np.std(pm_data), 2)
print("Standard Deviation: " + str(pm_std))
print()

print("Observations:")
print("We have to take into account that the raw data taken from the Andruino is not the data that we will use to evaluage the Particulate Matter in the air.")
print("For that we have to first process the that through an algorithm that will retrieve us the AQI (Air Quality Index).")
print("We can see that the lower element is "+str(min(pm_data))+", that value is repeated "+str(np.count_nonzero(pm_data == min(pm_data)))+" times.")
print("This is showing us the sensor has a lower limit there, and that lower value is the measure for when the sensor does not detect anything.")
print("We assume this because the lowest element is repeat many times with the exact same decimals.")
print()

print("Visualization:")
print("Mean and Standar Deviation")
df = pd.DataFrame({
      'data': pm_data,
      'mean': [pm_mean for i in range(len(pm_data))],
      'std': [np.std(pm_data) for i in range(len(pm_data))]})

df.plot()
plt.show()
print()

In [ ]:
# Particulate Matter Simulation Analysis
print("Particulate Matter Simulation:")
print("Simulation of decreasing for activation of Air Purifier")
print("Here we are going to propose a simulation case where we turn on the Air Purifier and the PM measure will decrease.")
print("Here we set a PURIFIER_FACTOR that is the factor by which the air is going to be purified in each measure.")
print("In each measure the air will have less particulate matter given by the factor (+-30%) and the previous accumulation of clean air.")
print("We will take the original data about particulate matter and we will take the amount relative to the air that was cleaned.")
print()

pm_data_air_purified = np.zeros_like(pm_data)
pm_data_air_purified_low = np.zeros_like(pm_data)
pm_data_air_purified_high = np.zeros_like(pm_data)
PURIFIER_FACTOR = 400
RND_VARIATION = 0.3
purified_air = 0
purified_air_low = 0
purified_air_high = 0
for i in range(pm_data.shape[0]):
  prfd_ft = PURIFIER_FACTOR * RND_VARIATION
  rnd_air_pur = random.uniform(PURIFIER_FACTOR - prfd_ft, PURIFIER_FACTOR + prfd_ft)
  purified_air = purified_air + rnd_air_pur
  result = pm_data[i] - purified_air
  if result > 0:
    pm_data_air_purified[i] = result
  else:
    pm_data_air_purified[i] = 0
    purified_air = 0

# Original mean and daviation for pm_data
print("Mean and Standar Deviation")
label_std = str(round(np.std(pm_data),2))
label_mean = str(round(np.mean(pm_data),2))
df = pd.DataFrame({
      'data ('+str(len(pm_data))+')': pm_data,
      'mean ('+label_mean+')': [np.mean(pm_data) for i in range(len(pm_data))],
      'std ('+label_std+')': [np.std(pm_data) for i in range(len(pm_data))]})

df.plot()
plt.show()
print()

# pm_data with air purified given PURIFIER_FACTOR
print("Mean and Standar Deviation for PURIFIER_FACTOR " + str(PURIFIER_FACTOR))
label_std = str(round(np.std(pm_data_air_purified),2))
label_mean = str(round(np.mean(pm_data_air_purified),2))
df = pd.DataFrame({
      'data ('+str(len(pm_data_air_purified))+')': pm_data_air_purified,
      'mean ('+label_mean+')': [np.mean(pm_data_air_purified) for i in range(len(pm_data_air_purified))],
      'std ('+label_std+')': [np.std(pm_data_air_purified) for i in range(len(pm_data_air_purified))]})

df.plot()
plt.show()
print()

print("Observations:")
print("We can see there is an considerable change from the graph with the air purified and the previous one.")
print("On top of that we can see the pronunciations of the spikes.")
print("The mean was reduced considerably ("+str(round(((np.mean(pm_data_air_purified)/np.mean(pm_data))-1)*-100,2))+"%) "
      "but that was not the case for the standar deviation ("+str(round(((np.std(pm_data_air_purified)/np.std(pm_data))-1)*-100,2))+"%).")
print("This is telling us that even though we can decrease de PM levels in general terms, handle the spikes of PM is out of hand.")
print("The evaluation give us very important information:")
print("    1- If we notice an spike is unsure that we will be able to purify the air in time to avoide health risk for the users.")
print("    2- This mesures are being taken in lapsus of thirty seconds. If we notice that after 30 sends the spike went down probably there is no danger")
print("       and the user is safe. But if we notice the spike remains, increases or goes down but not significantly it means the user is in danger.")
print("       But not only that, it means also that it is too late to take proper actions")
print("    3- For that reason we have to trigger an alarm right away (maybe with sounds or lights) to alert the users about their safety.")
print("       Maybe we could trigger another measure right away, like in miliseconds to verify the spike is decreasing fast enough to not be a risk.")
print("For the purpuse of this simulation we have put the sensors next to the oven, so is natural to see the spikes in the measures, and this give us a good idea ")
print("of a real danger case where the user health could be at risk.")
print()

In [6]:
# util AQI functions
def particle_count_to_pm(particle_count: float) -> float:
    """From a particle count per 0.01 cubic feet, calculate
    the estimated mass of PM2.5 in ug/m^3.

    Args:
        particle_count (float): Particle count per 0.01 cubic feet of air

    Returns:
        float: ug/m^3 of PM
    """
    particle_count = particle_count/0.0002831685 ## ratio of 0.01 cubic feet to cubic meter
    # expected particle mass = volume of a spherical particle with 1.75um diameter * unit density
    particle_volume = math.pi/6 * math.pow(1.75,3) # volume in cubic micrometers
    particle_density = 1.0 # 1 gram/cubic centimeter, want micrograms/cubic micrometer
    particle_density = particle_density/1000000 # 1e6 micrograms/1e12 cubic micrometers/cubic cm
    particle_mass = particle_density * particle_volume
    return particle_count * particle_mass

def pm_aqi(pm_measure: float)-> int:
    """Return the AQI for the measured pm2.5
    Based on method published here: https://www.airnow.gov/sites/default/files/2020-05/aqi-technical-assistance-document-sept2018.pdf

    Args:
        pm_measure (float): pm2.5 in micrograms/cubic meter. We reccommend taking a longer term  average
        here. This function only makes a point calculation

    Returns:
        float: AQI for measured PM
    """
    pm_measure = math.floor(pm_measure*10)/10 # truncate to 1 decimal place
    if pm_measure <= 12.0:
        return pm_aqi_breakpoint(pm_measure, 12.0, 0, 50, 0)
    elif pm_measure <= 35.4:
        return pm_aqi_breakpoint(pm_measure,35.4,12.1,100,51)
    elif pm_measure <= 55.4:
        return pm_aqi_breakpoint(pm_measure, 55.4,35.5,150,101)
    elif pm_measure <= 150.4:
        return pm_aqi_breakpoint(pm_measure, 150.4,55.5,200,151)
    elif pm_measure <= 250.4:
        return pm_aqi_breakpoint(pm_measure,250.4,150.5,300,201)
    elif pm_measure <= 500.4:
        return pm_aqi_breakpoint(pm_measure,500,5,250.5,500,301)
    else:
        return 501 # beyond the AQI

def pm_aqi_breakpoint(pm, bp_high, bp_low, index_high, index_low):
    index = ((index_high - index_low)/(bp_high - bp_low))*(pm - bp_low) + index_low
    if index - math.floor(index) > 0.5:
        return math.ceil(index)
    else:
        return math.floor(index)

def particle_count_to_aqi(particle_count: float) -> int:
    pm_measure = particle_count_to_pm(particle_count)
    aqi = pm_aqi(pm_measure)
    return aqi

In [ ]:
# AQI Analysis
print("The EPA developed the AQI, which reports levels of ozone, particle pollution, and other common air pollutants on the same scale.")
print("An AQI reading of 101 corresponds to a level above the national air quality standard - the higher the AQI rating, the greater the health impact.")
print("To know more about the AQI you can go to: https://docs.airnowapi.org/aq101")
print("For measuring the AQI we have follow the method used in this articule: https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0137789")
print("According to the paper they took two minute averages of measurements.")
print("Since our measures are being taken every thirdy seconds, we are going to average each four mesures as one to measure AQI.")
print("Then we are going to run an algorithm that will retrive us the PM in the air from the input we have from the sensors.")
print("Once we have that information we will compare it with the information received by the AirNowAPI organization.")
print()

# Average each four measures as one for AQI calculation
avg_end = len(pm_data) - (len(pm_data) % 4)
pm_data_avg = np.average(pm_data[0:avg_end].reshape(-1, 4), axis=1)
print("Raw input data after average every four observations:")
print(pm_data_avg)
print()

print("Mean and Standar Deviation")
label_std = str(round(np.std(pm_data_avg),2))
label_mean = str(round(np.mean(pm_data_avg),2))
df = pd.DataFrame({
      'data ('+str(len(pm_data_avg))+')': pm_data_avg,
      'mean ('+label_mean+')': [np.mean(pm_data_avg) for i in range(len(pm_data_avg))],
      'std ('+label_std+')': [np.std(pm_data_avg) for i in range(len(pm_data_avg))]})

df.plot()
plt.show()
print()

# We take the AQI values from the new data:
pm_data_aqi = np.zeros(pm_data_avg.shape[0])
for i in range(pm_data_aqi.shape[0]):
  pm_data_aqi[i] = particle_count_to_pm(pm_data_avg[i])

print("Raw AQI data:")
print(pm_data_aqi)
print()

print("AQI: Mean and Standar Deviation")
label_std = str(round(np.std(pm_data_aqi),2))
label_mean = str(round(np.mean(pm_data_aqi),2))
df = pd.DataFrame({
      'data ('+str(len(pm_data_aqi))+')': pm_data_aqi,
      'mean ('+label_mean+')': [np.mean(pm_data_aqi) for i in range(len(pm_data_aqi))],
      'std ('+label_std+')': [np.std(pm_data_aqi) for i in range(len(pm_data_aqi))]})

df.plot()
plt.show()
print()

print("These are the measures for AQI categories given their effects in health: ")
print("0-50: Good; 51-100: Moderate, 101-150: Unhealthy for Sensitive Groups, 151-200: Unhealthy, 201-300: Very Unhealthy, 301-500: Hazardous.")
print("Considering this fact we will graph an histogram with this distributions.")
print()

print("Histogram of AQI")
pm_data_aqi_rounded = np.around(pm_data_aqi)

plt.hist(pm_data_aqi_rounded, bins=range(int(min(pm_data_aqi_rounded)), int(max(pm_data_aqi_rounded))))
plt.show()
print()

print("Observations:")
print("All the measures are in the good AQI range for health.")
print("Even though we put the sensors next to a kitchen while we were using it, almost all the measures are inside the 0-4 range.")
print()

# After get ours PMs AQI measures we will call airnowapi API to compare with outside region data
print("Comparation with outside:")
print("Now we will call airnowapi API to see what are the values for that region.")
print("We are using the zipcode from the person that has the sensors in his home.")

API_KEY = '8BB7CB2F-3314-44DE-A612-A8FF83ECF6D2'
format = 'text/csv'
zipCode = '02143'
distance = '25'
today = date.today().strftime("%Y-%m-%d")
url = 'https://www.airnowapi.org/aq/forecast/zipCode/?format='+format+'&zipCode='+zipCode+'&date='+today+'&distance='+distance+'&API_KEY='+API_KEY
print("This is the endpoint we are calling: ")
print(url)
print()

r = requests.get(url)
zipCodeData = r.text
print("Airnowapi API raw data:")
print(zipCodeData)

pm2_dot_5_start = zipCodeData.find('PM2.5')+8
pm2_dot_5_end = zipCodeData[pm2_dot_5_start:].find(',') + pm2_dot_5_start -1
pm2_dot_5 = zipCodeData[pm2_dot_5_start:pm2_dot_5_end]
print("The PM2.5 of this area is: " + pm2_dot_5)
ratio_pm = round(np.mean(pm_data_aqi)/float(pm2_dot_5),2)
print("Taking the mean of our PM measures, the relation with the outside measure is: " + str(ratio_pm))
if np.mean(pm_data_aqi) > float(pm2_dot_5):
  print("This means the air has PM inside the house which is dangerous and a bad sign.")
  print("We will trigger the air purifiers but in this case just open the windows will help as well.")
else:
  print("This means the air inside the house is cleaner than outside.")
  print("Depending on the AQI value it is posible that we still have to trigger the air purifiers.")

In [ ]:
# Dashboard
print("As dashboard we decided to create an idea of how it will look and experience using colab.")
print("This is mainly because the course is not focus in web design, and we prefered to take all the advantage that we could have ")
print("from colab and python using the techniques and resourse that we have seen in the classes.")
print("Here is a simil of how the dashboard will look like:")
print()

# First layer: Templerature, Relative Humidity, PM
df1 = pd.DataFrame({
      'data ('+str(len(temp_data))+')': temp_data,
      'mean ('+str(round(np.mean(temp_data),2))+'C)': [np.mean(temp_data) for i in range(len(temp_data))],
      'std ('+str(round(np.std(temp_data),2))+'C)': [np.std(temp_data) for i in range(len(temp_data))]})

df2 = pd.DataFrame({
      'data ('+str(len(hum_data))+')': hum_data,
      'mean ('+str(round(np.mean(hum_data),2))+'%)': [np.mean(hum_data) for i in range(len(hum_data))],
      'std ('+str(round(np.std(hum_data),2))+'%)': [np.std(hum_data) for i in range(len(hum_data))]})

df3 = pd.DataFrame({
      'data ('+str(len(pm_data))+')': pm_data,
      'mean ('+str(round(np.mean(pm_data),2))+')': [np.mean(pm_data) for i in range(len(pm_data))],
      'std ('+str(round(np.std(pm_data),2))+')': [np.std(pm_data) for i in range(len(pm_data))]})

fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(16,4))  # 1 row, 3 columns
df1.plot(title="Temperature", ax=ax1)
df2.plot(title="Relative Humidity", ax=ax2)
df3.plot(title="Particulate Matter", ax=ax3)
plt.tight_layout()

df4 = pd.DataFrame({
      'data ('+str(len(pm_data_aqi))+')': pm_data_aqi,
      'mean ('+label_mean+')': [np.mean(pm_data_aqi) for i in range(len(pm_data_aqi))],
      'std ('+label_std+')': [np.std(pm_data_aqi) for i in range(len(pm_data_aqi))]})

fig, (ax4, ax5) = plt.subplots(1, 2, figsize=(16,4))  # 1 row, 2 columns
df4.plot(title="AQI", ax=ax4)
plt.title("AQI Histogram - ("+pm2_dot_5+"PM2.5 were registered outside)")
plt.hist(pm_data_aqi_rounded, bins=range(int(min(pm_data_aqi_rounded)), int(max(pm_data_aqi_rounded))))
plt.tight_layout()

# Dashboard alarms
tts = gTTS("All measures are good")
sound_file = 'warnings.wav'
tts.save(sound_file)

hum_data[len(hum_data)-1] = 95 # uncomment for testing
message = ""
if hum_data[len(hum_data)-1] >= 95:
  message = "Warning!, relative humidity is above 95%. The dust sensor will not be able to work in this conditions."

tmp_msg = ""
pm_data_aqi_mean = np.mean(pm_data_aqi)
pm_data_aqi_mean = 234 # uncomment for testing
if pm_data_aqi_mean > 100:
  message = message + "Warning!, particulate matters are too high."
  tmp_msg = "The air is unhealthy for sensitive groups."
  if pm_data_aqi_mean > 150:
    tmp_msg = "The air is unhealthy for everyone."
  if pm_data_aqi_mean > 200:
    tmp_msg = "The air is very unhealthy."
  if pm_data_aqi_mean > 300:
    tmp_msg = "The air is hazardous."
  message = message + tmp_msg

pm_data_aqi_mean = 234 # uncomment for testing
if pm_data_aqi_mean > 50 and pm_data_aqi_mean > float(pm2_dot_5):
  message = message + "Warning!, inside particulate matters are higher than outside. We recommend to open the windows"

if message != "":
  tts = gTTS(message)
  tts.save('warnings.wav')
Audio(sound_file, autoplay=True)

In [ ]:
# Other resources that we have recolected during the investigation but we did not use

# # Get data from RSS in HTML format
# print("API CALLS -------------------------------------------------------------------- ")
# print("Example of RSS call: California real time data (to compare with): https://feeds.airnowapi.org/rss/realtime/382.xml")
# url = 'https://feeds.airnowapi.org/rss/realtime/382.xml'
# r = requests.get(url)
# d = BeautifulSoup(r.text, 'xml')
# t = d.find_all('description')[1].text
# display(HTML(t))

# # Get data as CSV by zipcode
# print("API call for zipcode area")
# API_KEY = '8BB7CB2F-3314-44DE-A612-A8FF83ECF6D2'
# format = 'text/csv'
# zipCode = '20002'
# distance = '25'
# today = date.today().strftime("%Y-%m-%d")

# url = 'https://www.airnowapi.org/aq/forecast/zipCode/?format='+format+'&zipCode='+zipCode+'&date='+today+'&distance='+distance+'&API_KEY='+API_KEY
# r = requests.get(url)
# zipCodeData = r.text
# print(zipCodeData)

# # Get data as CSV by Latitude and Longitude
# print("API call for Latitude and Longitude")
# lat = '39.0509'
# lon = '-121.4453'

# url = 'https://www.airnowapi.org/aq/forecast/latLong/?format='+format+'&latitude='+lat+'&longitude='+lon+'&date='+today+'&distance='+distance+'&API_KEY='+API_KEY
# r = requests.get(url)
# latLotData = r.text
# print(latLotData)

# # Most complete query that provides all sort of data by an area
# print('### GENERAL DATA QUERY - we cant fetch more than a few hours ###')
# startDate = datetime.now().strftime("%Y-%m-%dT%H")
# endDate = (datetime.now() + timedelta(hours=1)).strftime("%Y-%m-%dT%H")
# elements = 'OZONE,PM25,PM10,CO,NO2,SO2'
# area = '-100.205070,30.716781,-95.337882,35.419415'
# dataType = 'B' # A: AQI, B: AQI + Concentrations, C: Concentrations
# verbose = str(0);
# monitorType = str(2); # 0: Permanent monitors only, 1: Mobile monitros only, 2: All monitors
# ARC = str(1)

# url = ('https://www.airnowapi.org/aq/data/?startDate='+
# startDate+'&endDate='+endDate+'&parameters='+elements+
# '&BBOX='+area+'&dataType='+dataType+'&format='+format+
# '&verbose='+verbose+'&monitorType='+monitorType+
# '&includerawconcentrations='+ARC+'&API_KEY='+API_KEY)
# r = requests.get(url)
# generalData = r.text
# print(generalData)

# print("END API CALLS -------------------------------------------------------------------- ")

# print("GRAPHS -------------------------------------------------------------------- ")

# print("Linear inside low-high pollution area")

# plt.style.use('_mpl-gallery')
# # make data
# np.random.seed(1)
# x = np.linspace(0, 8, 16)
# y1 = 3 + 4*x/8 + np.random.uniform(0.0, 0.5, len(x))
# y2 = 1 + 2*x/8 + np.random.uniform(0.0, 0.5, len(x))
# # plot
# fig, ax = plt.subplots()
# ax.fill_between(x, y1, y2, alpha=.5, linewidth=0)
# ax.plot(x, (y1 + y2)/2, linewidth=2)
# ax.set(xlim=(0, 8), xticks=np.arange(1, 8),
#        ylim=(0, 8), yticks=np.arange(1, 8))
# plt.show()


# print("Linear inside low-high pollution area - Second Option")

# plt.style.use('_mpl-gallery')

# # make data
# x = np.arange(0, 10, 2)
# ay = [1, 1.25, 2, 2.75, 3]
# by = [1, 1, 1, 1, 1]
# cy = [2, 1, 2, 1, 2]
# y = np.vstack([ay, by, cy])

# # plot
# fig, ax = plt.subplots()

# ax.stackplot(x, y)

# ax.set(xlim=(0, 8), xticks=np.arange(1, 8),
#        ylim=(0, 8), yticks=np.arange(1, 8))

# plt.show()


# print("Histogram with modals of different pollutions")

# plt.style.use('_mpl-gallery')

# #timestamp, temperature, relative humidity, particle count
# #31559,27.09,73.43,317.93
# #62145,27.19,72.53,0.62
# #92577,27.22,73.41,67.96
# #123375,27.25,73.03,168.76
# #153961,27.25,72.71,0.62
# #184546,27.21,72.61,0.62
# sensorTemp = np.array([27.09, 27.19, 27.22, 27.25, 27.25, 27.21])
# sensorPM = np.around(np.array([317.93, 0.62, 67.96, 168.76, 0.62, 0.62]) / 100)


# # plot:
# fig, ax = plt.subplots()

# ax.hist(sensorPM, bins=3, linewidth=0.5, edgecolor="white")

# ax.set(xlim=(0, 3), xticks=np.arange(0, 3),
#        ylim=(0, 5), yticks=np.linspace(0, 5, 6))

# plt.show()


# print("Histogram 2d?")

# plt.style.use('_mpl-gallery-nogrid')

# # make data: correlated + noise
# np.random.seed(1)
# x = np.random.randn(5000)
# y = 1.2 * x + np.random.randn(5000) / 3

# # plot:
# fig, ax = plt.subplots()

# ax.hist2d(x, y, bins=(np.arange(-3, 3, 0.1), np.arange(-3, 3, 0.1)))

# ax.set(xlim=(-2, 2), ylim=(-3, 3))

# plt.show()


# print("Scatter ?")


# plt.style.use('_mpl-gallery')

# # make the data
# np.random.seed(3)
# x = 4 + np.random.normal(0, 2, 24)
# y = 4 + np.random.normal(0, 2, len(x))
# # size and color:
# sizes = np.random.uniform(15, 80, len(x))
# colors = np.random.uniform(15, 80, len(x))

# # plot
# fig, ax = plt.subplots()

# ax.scatter(x, y, s=sizes, c=colors, vmin=0, vmax=100)

# ax.set(xlim=(0, 8), xticks=np.arange(1, 8),
#        ylim=(0, 8), yticks=np.arange(1, 8))

# plt.show()

# print("LINEAR REGRESSION: https://colab.research.google.com/github/jakevdp/PythonDataScienceHandbook/blob/master/notebooks/05.06-Linear-Regression.ipynb")
# print("MULTI LINEAR REGRESSION: https://colab.research.google.com/github/rafiag/DTI2020/blob/main/002a_Multi_Linear_Regression_(EN).ipynb")



# Nueva sección